# BLOCK 1: Load & Explore the Dataset

In [1]:
from datasets import load_dataset

# Load the LIAR dataset from HuggingFace
dataset = load_dataset("liar")

# Check available splits
print(dataset)

# View a few examples
print(dataset["train"][0])

DatasetDict({
    train: Dataset({
        features: ['id', 'label', 'statement', 'subject', 'speaker', 'job_title', 'state_info', 'party_affiliation', 'barely_true_counts', 'false_counts', 'half_true_counts', 'mostly_true_counts', 'pants_on_fire_counts', 'context'],
        num_rows: 10269
    })
    test: Dataset({
        features: ['id', 'label', 'statement', 'subject', 'speaker', 'job_title', 'state_info', 'party_affiliation', 'barely_true_counts', 'false_counts', 'half_true_counts', 'mostly_true_counts', 'pants_on_fire_counts', 'context'],
        num_rows: 1283
    })
    validation: Dataset({
        features: ['id', 'label', 'statement', 'subject', 'speaker', 'job_title', 'state_info', 'party_affiliation', 'barely_true_counts', 'false_counts', 'half_true_counts', 'mostly_true_counts', 'pants_on_fire_counts', 'context'],
        num_rows: 1284
    })
})
{'id': '2635.json', 'label': 0, 'statement': 'Says the Annies List political group supports third-trimester abortions on deman

# BLOCK 2: Preprocess — Label Mapping & Tokenization

## Step 1: Confirm Label Mapping

In [2]:
id2label = {
    0: "pants-fire",
    1: "false",
    2: "barely-true",
    3: "half-true",
    4: "mostly-true",
    5: "true"
}
label2id = {v: k for k, v in id2label.items()}

## Step 2: Tokenize the “statement” column

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize_function(example):
    return tokenizer(
        example["statement"],
        padding="max_length",  # or "longest" during testing
        truncation=True,
        max_length=128
    )

## Step 3: Apply Tokenizer Across Dataset

In [4]:
# Apply to all splits
tokenized_datasets = dataset.map(tokenize_function, batched=True)

This adds:
	•	input_ids
	•	attention_mask
	•	Keeps label as is

# BLOCK 3: Prepare the Dataset for HuggingFace Trainer

## Step 1: Keep Only Required Columns

The model only needs:
	•	input_ids
	•	attention_mask
	•	label, So we’ll remove all other columns:

In [5]:
tokenized_datasets = tokenized_datasets.remove_columns(
    [col for col in tokenized_datasets["train"].column_names if col not in ["input_ids", "attention_mask", "label"]]
)

## Step 2: Set Dataset Format for PyTorch

In [6]:
tokenized_datasets.set_format("torch")

## Step 3: Split into Train/Validation/Test

In [7]:
train_dataset = tokenized_datasets["train"]
eval_dataset = tokenized_datasets["validation"]
test_dataset = tokenized_datasets["test"]

This clean split ensures:
	•	train_dataset → for model learning
	•	eval_dataset → for validation during training
	•	test_dataset → for final evaluation after training

# BLOCK 4: Load DistilBERT Model for 6-Class Classification

## Step 1: Define Label Mappings

In [8]:
id2label = {
    0: "pants-fire",
    1: "false",
    2: "barely-true",
    3: "half-true",
    4: "mostly-true",
    5: "true"
}
label2id = {v: k for k, v in id2label.items()}

## Step 2: Load the Model

In [9]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=6,
    id2label=id2label,
    label2id=label2id
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# BLOCK 5: Define TrainingArguments & Trainer

## Step 1: Define TrainingArguments

In [10]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",  # Updated parameter name
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=100,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)

## Step 2: Define a Metric Function

In [12]:
import numpy as np
import evaluate

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return accuracy.compute(predictions=predictions, references=labels)

## Step 3: Create the Trainer

In [13]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)

/var/folders/hv/375whh5x1pldcb3r78dqhr2h0000gn/T/ipykernel_36227/1757859548.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


## Step: BLOCK 6 — Launch Training

In [ ]:
trainer.train()

This will:
	•	Start fine-tuning DistilBERT on your LIAR dataset
	•	Show training + validation loss
	•	Save model checkpoints

## Step 2: Save the Fine-Tuned Model & Tokenizer

In [ ]:
model_save_path = "../models/fine_tuned_distilbert_liar"

# Save model & tokenizer
trainer.save_model(model_save_path)
tokenizer.save_pretrained(model_save_path)

import shutil

# Zip the folder
shutil.make_archive(model_save_path, 'zip', model_save_path)

In [ ]:
from google.colab import files

# Download
files.download(f"{model_save_path}.zip")

# BLOCK 7: Evaluate Fine-Tuned Model on Test Set

## Step 1: Evaluate Using trainer.evaluate()

In [ ]:
results = trainer.evaluate(eval_dataset=test_dataset)
print("📊 Test Set Evaluation Metrics:", results)

This gives you:
	•	eval_loss
	•	eval_accuracy
	•	(More, if additional metrics are added like F1)

## Step 2: Classification Report & Confusion Matrix

In [ ]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Get predictions
predictions = trainer.predict(test_dataset)
y_true = predictions.label_ids
y_pred = np.argmax(predictions.predictions, axis=1)

# Print classification report
print(classification_report(y_true, y_pred, target_names=list(id2label.values())))

# Plot confusion matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt="d", xticklabels=id2label.values(), yticklabels=id2label.values(), cmap="Blues")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix - Test Set")
plt.show()